In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy.io import loadmat
import visionloader as vl
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
from scipy.optimize import minimize, differential_evolution
import os
import src.fitting as fitting
# testing testing

# Load in triplet stim elecResps and amplitudes

In [2]:
dataset = '2021-05-27-4/data003/'
electrical_path = '/Volumes/Analysis/' + dataset

In [3]:
vis_datapath = '/Volumes/Analysis/2021-05-27-4/data001'
vis_datarun = 'data001'
vcd = vl.load_vision_data(vis_datapath, vis_datarun,
                          include_neurons=True,
                          include_ei=True,
                          include_params=True,
                          include_noise=True)

coords = vcd.get_electrode_map()

In [4]:
patterns = np.array([33, 35, 40])
n = 2764

In [5]:
all_elecs = []
neg_inds_total = 0
pos_inds_total = 0

for i in range(len(patterns)):
    p = patterns[i]
    print('Pattern ' + str(p))
    filepath = '/Volumes/Scratch/Users/praful/triplet_gsort_v2_30um_periphery-affinity/' + dataset + 'p' + str(p) + '/'

    triplet_elecs = (loadmat(electrical_path + 'pattern_files/p' + str(p) + '.mat', 
                             squeeze_me=True, struct_as_record=False)['patternStruct'].stimElecs)

    amplitudes = (loadmat(electrical_path + 'pattern_files/p' + str(p) + '.mat', 
                             squeeze_me=True, struct_as_record=False)['patternStruct'].amplitudes)
        
    num_pts = len(amplitudes)

    triplet_probs = np.zeros(num_pts)
    for k in range(num_pts):
        with open(filepath + 'gsort_tri_v2_n'+str(n)+'_p'+str(p)+'_k'+str(k)+'.pkl', 'rb') as f:
            prob_dict = pickle.load(f)
            triplet_probs[k] = prob_dict['prob']

    neg_inds = np.where((np.all(amplitudes < 0, axis=1)) & (triplet_probs > 0.08))[0]
    pos_inds = np.where((np.all(amplitudes > 0, axis=1)) & (triplet_probs > 0.08))[0]

    all_elecs.append(triplet_elecs)
        
    pos_inds_total += len(pos_inds)
    neg_inds_total += len(neg_inds)

Pattern 33
Pattern 35
Pattern 40


In [6]:
pos_inds_total, neg_inds_total

(26, 50)

In [7]:
all_elecs_array = np.unique(np.array(all_elecs))

In [8]:
all_elecs_array

array([206, 207, 216, 217, 226], dtype=uint8)

In [9]:
X_pos = []
y_pos = []

X_neg = []
y_neg = []

trials_pos = []
trials_neg = []

In [10]:
for i in range(len(patterns)):
    p = patterns[i]
    print('Pattern ' + str(p))
    filepath = '/Volumes/Scratch/Users/praful/triplet_gsort_v2_30um_periphery-affinity/' + dataset + 'p' + str(p) + '/'

    triplet_elecs = (loadmat(electrical_path + 'pattern_files/p' + str(p) + '.mat', 
                             squeeze_me=True, struct_as_record=False)['patternStruct'].stimElecs)

    amplitudes = (loadmat(electrical_path + 'pattern_files/p' + str(p) + '.mat', 
                             squeeze_me=True, struct_as_record=False)['patternStruct'].amplitudes)
        
    num_pts = len(amplitudes)

    triplet_probs = np.zeros(num_pts)
    triplet_trials = np.zeros(num_pts, dtype=int)
    for k in range(num_pts):
        with open(filepath + 'gsort_tri_v2_n'+str(n)+'_p'+str(p)+'_k'+str(k)+'.pkl', 'rb') as f:
            prob_dict = pickle.load(f)
            triplet_probs[k] = prob_dict['prob']
            triplet_trials[k] = prob_dict['num_trials']

    neg_inds = np.where((np.all(amplitudes < 0, axis=1)) & (triplet_probs > 0.08))[0]
    pos_inds = np.where((np.all(amplitudes > 0, axis=1)) & (triplet_probs > 0.08))[0]
    
    elec_inds = np.searchsorted(all_elecs_array, triplet_elecs)
    
    y = triplet_probs[pos_inds]
    X = amplitudes[pos_inds]
    trials = triplet_trials[pos_inds]
    X_pos_p = np.zeros((len(pos_inds), len(all_elecs_array)))
    X_pos_p[:, elec_inds] = X
    X_pos.append(X_pos_p)
    y_pos.append(y)
    trials_pos.append(trials)
    
    y = triplet_probs[neg_inds]
    X = amplitudes[neg_inds]
    trials = triplet_trials[neg_inds]
    X_neg_p = np.zeros((len(neg_inds), len(all_elecs_array)))
    X_neg_p[:, elec_inds] = X
    X_neg.append(X_neg_p)
    y_neg.append(y)
    trials_neg.append(trials)
    
    print(all_elecs_array)
    print(triplet_elecs)
    print(elec_inds)
    print(amplitudes[neg_inds])
    print(X_neg_p)
    print(y)
    print(trials)

Pattern 33
[206 207 216 217 226]
[216 207 206]
[2 1 0]
[[-0.28125 -0.28125 -0.09375]
 [-0.84375 -1.21875 -1.78125]
 [-0.46875 -1.21875 -0.65625]
 [-1.21875 -1.21875 -0.28125]
 [-1.78125 -0.28125 -1.59375]
 [-0.84375 -0.65625 -1.40625]
 [-1.40625 -1.78125 -0.09375]
 [-1.40625 -0.84375 -0.65625]
 [-1.03125 -1.21875 -1.59375]
 [-0.65625 -1.40625 -1.78125]]
[[-0.09375 -0.28125 -0.28125  0.       0.     ]
 [-1.78125 -1.21875 -0.84375  0.       0.     ]
 [-0.65625 -1.21875 -0.46875  0.       0.     ]
 [-0.28125 -1.21875 -1.21875  0.       0.     ]
 [-1.59375 -0.28125 -1.78125  0.       0.     ]
 [-1.40625 -0.65625 -0.84375  0.       0.     ]
 [-0.09375 -1.78125 -1.40625  0.       0.     ]
 [-0.65625 -0.84375 -1.40625  0.       0.     ]
 [-1.59375 -1.21875 -1.03125  0.       0.     ]
 [-1.78125 -1.40625 -0.65625  0.       0.     ]]
[0.1        0.15789474 0.15789474 0.42105263 0.10526316 0.25
 0.10526316 0.78947368 0.35       0.35      ]
[20 19 19 19 19 20 19 19 20 20]
Pattern 35
[206 207 216 

In [11]:
all_amps_neg = np.vstack(X_neg)
all_probs_neg = np.hstack(y_neg)
trials_neg = np.hstack(trials_neg)

In [12]:
all_amps_pos = np.vstack(X_pos)
all_probs_pos = np.hstack(y_pos)
trials_pos = np.hstack(trials_pos)

In [13]:
all_amps_neg

array([[-0.09375, -0.28125, -0.28125,  0.     ,  0.     ],
       [-1.78125, -1.21875, -0.84375,  0.     ,  0.     ],
       [-0.65625, -1.21875, -0.46875,  0.     ,  0.     ],
       [-0.28125, -1.21875, -1.21875,  0.     ,  0.     ],
       [-1.59375, -0.28125, -1.78125,  0.     ,  0.     ],
       [-1.40625, -0.65625, -0.84375,  0.     ,  0.     ],
       [-0.09375, -1.78125, -1.40625,  0.     ,  0.     ],
       [-0.65625, -0.84375, -1.40625,  0.     ,  0.     ],
       [-1.59375, -1.21875, -1.03125,  0.     ,  0.     ],
       [-1.78125, -1.40625, -0.65625,  0.     ,  0.     ],
       [ 0.     , -1.21875, -1.03125, -0.28125,  0.     ],
       [ 0.     , -1.78125, -0.65625, -1.40625,  0.     ],
       [ 0.     , -1.59375, -0.09375, -1.59375,  0.     ],
       [ 0.     , -1.03125, -1.40625, -0.84375,  0.     ],
       [ 0.     , -0.65625, -0.46875, -0.65625,  0.     ],
       [ 0.     , -1.40625, -0.46875, -0.84375,  0.     ],
       [ 0.     , -1.40625, -1.21875, -1.21875,  0.     

In [14]:
all_amps_pos.shape, all_probs_pos.shape, trials_pos.shape

((26, 5), (26,), (26,))

In [17]:
degree = 1
multi_X, multi_y = fitting.convertToBinaryClassifier(all_probs_neg, trials_neg, all_amps_neg, degree)

In [18]:
multi_X.shape, multi_y.shape

((1080, 6), (1080,))

In [ ]:
def negLL(params, *args):
    X, y, verbose, method = args
    
    w = params
    
    # Get predicted probability of spike using current parameters
    yPred = 1 / (1 + np.exp(-X @ w))
    yPred[yPred == 1] = 0.999999     # some errors when yPred is exactly 1 due to taking log(1 - 1)

    # Calculate negative log likelihood
    NLL = -np.sum(y * np.log2(yPred) + (1 - y) * np.log2(1 - yPred))     # negative log likelihood for logistic

    # if method == 'MAP':
    #     penalty = 0.5 * (w - mu) @ np.linalg.inv(cov) @ (w - mu)     # penalty term according to MAP regularization
    # else:
    #     penalty = 0
    penalty = 0
    if verbose:
        print(NLL, penalty)
    return(NLL + penalty)

In [ ]:
def fsigmoid(X, w):
    return 1.0 / (1.0 + np.exp(-X @ w))

In [ ]:
mu = np.concatenate((np.array([1]), np.zeros(multi_X.shape[-1]-1)))
multi_results = minimize(negLL, x0=mu, args=(multi_X, multi_y, False, 'none'))
multi_weights = multi_results.x

In [ ]:
multi_weights

In [ ]:
poly = PolynomialFeatures(degree)
test_X = poly.fit_transform(all_amps_neg)
test_y = all_probs_neg

plt.scatter(test_X @ multi_weights, test_y)
sigmoid_x = np.linspace(-4, 4, 100)
plt.plot(sigmoid_x, 1/(1 + np.exp(-sigmoid_x)))